In [1]:
# from keras.models import Sequential
# from keras.layers.core import Dense, Dropout, Activation, Flatten
# from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
# from keras.optimizers import SGD, Adadelta, Adagrad
# from keras.utils import np_utils, generic_utils
# from keras.callbacks import EarlyStopping
# from keras.layers.advanced_activations import PReLU, LeakyReLU
# from keras.layers import Embedding,GRU,TimeDistributed,RepeatVector,Merge
# from keras.preprocessing.text import one_hot
# from keras.preprocessing import sequence

import numpy as np
# from vgg16 import Vgg16

# from os import listdir
# from os.path import isfile, join

# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg

# import PIL.Image

# import json
# from tqdm import tqdm

# from keras.optimizers import SGD, RMSprop, Adam

from utils import *
# import cPickle as pickle
# from matplotlib import pyplot as plt

import collections

import preprocessing as preproc

Using Theano backend.
Using gpu device 0: GeForce GTX 1080 (CNMeM is disabled, cuDNN 5110)
/opt/conda/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
base_path = full_data_path
train_path = base_path + train_folder
val_path = base_path + val_folder


In [3]:
def first_N_captions_sum(captions,N):
    caption_lengths = [len(caption.split()) for caption in captions]
    counter=collections.Counter(caption_lengths)
    return sum([value for key,value in counter.iteritems() if key <= N])

def get_unique_words(captions):
    unique_words = []
    words = [caption.split() for caption in captions]
   
    for word in words:
        unique_words.extend(word)
        
    unique_words = list(set(unique_words))
    
    return unique_words

def no_words_with_min_appearances(captions,min_no_of_occurences):
    
    sentences = [caption.split() for caption in captions]
    words = []
    for word in sentences:
        words.extend(word)

    counter=collections.Counter(words)
    nr_words = len([_ for word,no_appearances in counter.most_common() if no_appearances >= min_no_of_occurences])
    
    return nr_words


In [4]:
train_captions = preproc.get_captions_from_batch(train_path + captions_folder, batch_nr = 0)
val_captions = preproc.get_captions_from_batch(val_path + captions_folder, batch_nr = 0)

captions = list(train_captions) + list(val_captions)
len(captions)

122050

# Number of captions having length less than X

In [5]:
totalSum = first_N_captions_sum(captions,500)

for N in [9,10,12,15,20,50]:
    firstN = first_N_captions_sum(captions,N)
    print("Captions of length <= %d  ----> Sum = %d  => Percentage = %f"%(N,firstN,firstN/float(totalSum)))
    

Captions of length <= 9  ----> Sum = 863  => Percentage = 0.007071
Captions of length <= 10  ----> Sum = 19621  => Percentage = 0.160762
Captions of length <= 12  ----> Sum = 73007  => Percentage = 0.598173
Captions of length <= 15  ----> Sum = 112702  => Percentage = 0.923408
Captions of length <= 20  ----> Sum = 121009  => Percentage = 0.991471
Captions of length <= 50  ----> Sum = 122049  => Percentage = 0.999992


# Number of words having more appearances than X

In [6]:
unique_words = preproc.load_obj(base_path + general_datastruct_folder + "unique_words")
allWords = float(len(unique_words))
allWords

22261.0

In [7]:
for N in [1,2,3,5,10,100]:
    nr_words = no_words_with_min_appearances(captions,N)
    print("Min_no = %d app --> No of words = %d  ---> Percentage = %f"%(N,nr_words,float(nr_words)/allWords))

N = 1
N = 2
nr_words = no_words_with_min_appearances(captions,N)
print("Min_no = %d app --> No of words = %d  ---> Percentage = %f"%(N,nr_words,float(nr_words)/allWords))

N = 3
nr_words = no_words_with_min_appearances(captions,N)
print("Min_no = %d app --> No of words = %d  ---> Percentage = %f"%(N,nr_words,float(nr_words)/allWords))

N = 5
nr_words = no_words_with_min_appearances(captions,N)
print("Min_no = %d app --> No of words = %d  ---> Percentage = %f"%(N,nr_words,float(nr_words)/allWords))

N = 10
nr_words = no_words_with_min_appearances(captions,N)
print("Min_no = %d app --> No of words = %d  ---> Percentage = %f"%(N,nr_words,float(nr_words)/allWords))

N = 100
nr_words = no_words_with_min_appearances(captions,N)
print("Min_no = %d app --> No of words = %d  ---> Percentage = %f"%(N,nr_words,float(nr_words)/allWords))

N = 500
nr_words = no_words_with_min_appearances(captions,N)
print("Min_no = %d app --> No of words = %d  ---> Percentage = %f"%(N,nr_words,float(nr_words)/allWords))

N = 5000
nr_words = no_words_with_min_appearances(captions,N)
print("Min_no = %d app --> No of words = %d  ---> Percentage = %f"%(N,nr_words,float(nr_words)/allWords))

Min_no = 1 app --> No of words = 22261  ---> Percentage = 1.000000
Min_no = 2 app --> No of words = 11622  ---> Percentage = 0.522079
Min_no = 3 app --> No of words = 8821  ---> Percentage = 0.396254
Min_no = 5 app --> No of words = 6286  ---> Percentage = 0.282377
Min_no = 10 app --> No of words = 4125  ---> Percentage = 0.185302
Min_no = 100 app --> No of words = 946  ---> Percentage = 0.042496


NameError: name 'n' is not defined

# Captions containing only common words

In [8]:
total_length = len(captions)

MIN_NO_OF_APP = 2
nr_captions =  np.sum(preproc.compute_common_words_caption_mask(captions,min_no_of_app = MIN_NO_OF_APP))
print("valid_captions_len = %d app --> min_no_on_app = %d  ---> Percentage = %f"%(nr_captions,MIN_NO_OF_APP,float(nr_captions)/total_length))

MIN_NO_OF_APP = 3
nr_captions =  np.sum(preproc.compute_common_words_caption_mask(captions,min_no_of_app = MIN_NO_OF_APP))
print("valid_captions_len = %d app --> min_no_on_app = %d  ---> Percentage = %f"%(nr_captions,MIN_NO_OF_APP,float(nr_captions)/total_length))

MIN_NO_OF_APP = 4
nr_captions =  np.sum(preproc.compute_common_words_caption_mask(captions,min_no_of_app = MIN_NO_OF_APP))
print("valid_captions_len = %d app --> min_no_on_app = %d  ---> Percentage = %f"%(nr_captions,MIN_NO_OF_APP,float(nr_captions)/total_length))

MIN_NO_OF_APP = 10
nr_captions =  np.sum(preproc.compute_common_words_caption_mask(captions,min_no_of_app = MIN_NO_OF_APP))
print("valid_captions_len = %d app --> min_no_on_app = %d  ---> Percentage = %f"%(nr_captions,MIN_NO_OF_APP,float(nr_captions)/total_length))

MIN_NO_OF_APP = 50
nr_captions =  np.sum(preproc.compute_common_words_caption_mask(captions,min_no_of_app = MIN_NO_OF_APP))
print("valid_captions_len = %d app --> min_no_on_app = %d  ---> Percentage = %f"%(nr_captions,MIN_NO_OF_APP,float(nr_captions)/total_length))


valid_captions_len = 113093 app --> min_no_on_app = 2  ---> Percentage = 0.926612
valid_captions_len = 108947 app --> min_no_on_app = 3  ---> Percentage = 0.892642
valid_captions_len = 105651 app --> min_no_on_app = 4  ---> Percentage = 0.865637
valid_captions_len = 93777 app --> min_no_on_app = 10  ---> Percentage = 0.768349
valid_captions_len = 64546 app --> min_no_on_app = 50  ---> Percentage = 0.528849
